# Домашнее задание

### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
import pandas as pd
# "ratings.dat"
# UserID::MovieID::Rating::Timestamp
# "users.dat"
# UserID::Gender::Age::Occupation::Zip-code
# "movies.dat"
# MovieID::Title::Genres

In [7]:
def m_score(row):
    """
    функция расчета рейтинга
    """
    if row['Rating'] <= 2:
        return 'low'
    elif row['Rating'] <=4:
        return 'average'
    else:
        return 'high'

#готовим датасеты
ratings = pd.read_csv(filepath_or_buffer='ml-1m/ratings.dat', delimiter=r'::', header=None, names=['UserID','MovieID','Rating','Timestamp'], usecols=[1,2], dtype=int, engine='python')



In [53]:
#решение
rated = ratings.groupby(['MovieID']).mean()
rated['rate'] = result.apply(m_score, axis=1)

rated.head(10)

,Rating,rate
MovieID,,
1,4.146846,high
2,3.201141,average
3,3.016736,average
4,2.729412,average
5,3.006757,average
6,3.878723,average
7,3.410480,average
8,3.014706,average
9,2.656863,average


### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [24]:
#готовим датасеты
geo_data = {
            'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }

cities = {} #для ускорения поиска

for reg in geo_data.items():
    for city in reg[1]:
        cities[city] = reg[0]
        


keywords = pd.read_csv('keywords.csv')


In [25]:
#решение

import re


def set_region(row):
    """
    Поиск региона
    """
    words = re.split(r'\s', row['keyword'])
    for w in words:
        res = cities.get(w.lower()) # в идеале нужен нечеткий поиск - либо сложно, либо унылокод
        if res:
            return res
    #else:    
    res = 'undefined'
    return res
    
keywords['region'] = keywords.apply(set_region, axis=1)

keywords.head()


,keyword,shows,region
0,погода в москве,745745,undefined
1,сложно тестировать без города Псков,1337,Северо-Запад
2,вк,64292779,undefined
3,одноклассники,63810309,undefined
4,порно,41747114,undefined


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
        для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга


In [54]:
# "movies.dat"
# MovieID::Title::Genres

#rated - датасет из первого задания MovieID:Rating:rate
#ради интереса будем искать год через регулярное выражение, и в промежутке без полного списка
# - апдейт: а промежуток и не нужен
#years = [1950, 2010]

def set_year(row):
    """
    анализ года выпуска фильма из названия
    """
    year = re.search(r'\(\d{4}\)$', row['Title']) # (YYYY)$конецстроки
    if year:
        return int(year.group(0)[1:5]) #сюда можно дописать проверку на 1950-2010
    else:
        return 1900

#готовим датасеты
movies = pd.read_csv(filepath_or_buffer='ml-1m/movies.dat', delimiter=r'::', header=None, names=['MovieID','Title','Genres'], usecols=[0,1], engine='python')

In [84]:
#решение

movies['year'] = movies.apply(set_year, axis=1)

#здесь очень не хочится скрещивать датасеты руками, поэтому гуглим join

# res = movies.join(rated.set_index('MovieID'), on='MovieID', how='inner')
# res = movies.join(rated.set_index(rated.columns[0]), on=movies.columns[0], how='inner')
# в этот момент стало очень грустно, когда оказалось, что значащая колонка является индексом таблицы и по [0] или 'name' к ней не обратиться
# разработчики pandas явно на какой-то своей волне

res = movies.join(rated)

print(res.groupby('year').mean()[['Rating']].sort_values('Rating', ascending=False).head(20))


        Rating
year          
1939  3.896190
1938  3.778261
1946  3.702438
1950  3.699199
1921  3.666667
1947  3.648937
1949  3.607396
1952  3.592187
1941  3.538259
1937  3.518754
1953  3.498658
1955  3.494114
1990  3.474844
1922  3.469014
1983  3.459050
1988  3.440868
1979  3.427279
1954  3.418045
1926  3.405960
1956  3.401902
